# Aanalysis code in year 4

In [ ]:
import csv
import sqlite3

import numpy as np
import pandas as pd

In [ ]:
indexpath = 'dbv3-index.db'
corepath = 'dbv3-core.db'
servicepath = 'dbv3-service.db'
utilpath = 'dbv3-util.db'

In [ ]:
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.execute(f'''ATTACH DATABASE '{indexpath}' AS DBINDEX;''')
cur.execute(f'''ATTACH DATABASE '{corepath}' AS DBCORE;''')
cur.execute(f'''ATTACH DATABASE '{servicepath}' AS DBSERVICE;''')
cur.execute(f'''ATTACH DATABASE '{utilpath}' AS DBUTIL;''')
conn.commit()

In [ ]:
tagpath = 'datasets/NamedCluster-Year4.csv'
rootaddrpath = 'datasets/NamedCluster-Year4-RootAddress.csv'

In [ ]:
with open(tagpath, 'r') as f:
    reader = csv.DictReader(f)
    print(reader.fieldnames)

```sql
-- TODO: Cluster ID and Root Address by Address
SELECT MIN(DBSERVICE.Cluster.addr) AS clusterId, DBINDEX.AddrID.addr AS rootAddress
FROM DBSERVICE.Cluster
INNER JOIN DBINDEX.AddrID ON DBSERVICE.Cluster.addr = DBINDEX.AddrID.id
WHERE DBSERVICE.Cluster.cluster = (
    SELECT DBSERVICE.Cluster.cluster
    FROM DBSERVICE.Cluster
    WHERE DBSERVICE.Cluster.addr = (
        SELECT DBINDEX.AddrID.id
        FROM DBINDEX.AddrID
        WHERE DBINDEX.AddrID.addr = 'bc1q2lgzm0mh6qgydmc5vvauv2uh6e745yn797wwqs'));
```

In [ ]:
query = '''
SELECT MIN(DBSERVICE.Cluster.addr) AS clusterId, DBINDEX.AddrID.addr AS rootAddress
FROM DBSERVICE.Cluster
INNER JOIN DBINDEX.AddrID ON DBSERVICE.Cluster.addr = DBINDEX.AddrID.id
WHERE DBSERVICE.Cluster.cluster = (
    SELECT DBSERVICE.Cluster.cluster
    FROM DBSERVICE.Cluster
    WHERE DBSERVICE.Cluster.addr = (
        SELECT DBINDEX.AddrID.id
        FROM DBINDEX.AddrID
        WHERE DBINDEX.AddrID.addr = ?));
'''

In [ ]:
rows = []
with open(tagpath, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cur.execute(query, (row['address'],))
        res = cur.fetchone()
        rows.append((row['categoryName'], row['clusterName'], row['address'], res[1], res[0]))
print(len(rows))

In [ ]:
df = pd.DataFrame(rows, columns=('categoryName', 'clusterName', 'address', 'rootAddress', 'clusterId'))
df

In [ ]:
df.to_csv(rootaddrpath)

In [ ]:
df.group_by((